In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

In [ ]:
import cv2

# Predict results using U-net model

## Step 1: Crop the original images to small pieces, in total there are 52 samples

## Step 2: Read the crop images for one sample,  then upsample them to the shape for the network, in this example, the target size is 256 x 256

## Step 3: Create prediction results of one sample 

## Step 4: Stitch it back to a whole images

In [ ]:
def read_crop_write(read_path, write_path, N_pieces): 
    #images = []
    for filename in os.listdir(read_path):
        f = os.path.join(read_path,filename)
        img =cv2.imread(f, cv2.IMREAD_GRAYSCALE)

        I_crop_v  =  np.asarray(np.split(img, 32, axis = 0)) 
        # when split in this axis can be recovered by using I_crop_recover = I_crop_v.reshape(1024,1024)
        I_crop_vv = np.asarray(np.split(I_crop_v, 32, axis = 2))

        for i in range(0,int(sqrt(N_pieces))):
            for j in range (0,int(sqrt(N_pieces))):
                prefix= re.sub("\D", "", filename)
                filename_new = write_path + '\\'+ prefix + '_'+ str(i)+'_'+str(j) +'.jpg'
                cv2.imwrite(filename_new, I_crop_vv[j, i, :, :])



In [ ]:
from math import sqrt
def create_filename(N_slices, N_image_start, N_image_stop): 
    # input: 
    # N_slices -  number of slice per images
    # N_image_start - index of the first image to read
    # N_image_stop - index of the last image to read
    
    N_slices = int(sqrt(N_slices))
    filename = []
    for i in range(N_image_start, N_image_stop+1):
        for j in range(0, N_slices):
            for k in range(0, N_slices):                  
                f = str(i)+'_'+str(j)+'_'+str(k)+'.jpg'
                                   
                filename.append(f)
    #print(filename)
    return (filename)

In [ ]:
def testGenerator(test_path,N_slices, N_image_start, N_image_stop,target_size = (256,256),flag_multi_class = False,as_gray = True):
    # input:
    # test_path: path of the images to read
    # N_slices -  number of slice per images
    # N_image_start - index of the first image to read
    # N_image_stop - index of the last image to read 
    # target_size - shape of the images for output
    file_list = create_filename(N_slices, N_image_start, N_image_stop)
    for filename in file_list:
        f = os.path.join(test_path,filename)
        img =io.imread(f, as_gray = as_gray)

        img = img / 255
        img = trans.resize(img,target_size)
        img = np.reshape(img,img.shape+(1,)) if (not flag_multi_class) else img
        img = np.reshape(img,(1,)+img.shape)
        yield img

In [ ]:
def saveResult(save_path,npyfile,flag_multi_class = False,num_class = 2):
    for i,item in enumerate(npyfile):
        img = labelVisualize(num_class,COLOR_DICT,item) if flag_multi_class else item[:,:,0]
        io.imsave(os.path.join(save_path,"%d_predict.jpg"%i),img)

In [ ]:
def stitch_image (images, slices):
    block_row = int(sqrt(slices))
    for j in range(0,block_row):
        output = images[j*block_row,:,:]
        for i in range(1,block_row):
            output = np.block([output, images[j*block_row+i,:,:]])
        if j == 0:
            output_whole=output
        else:
            output_whole = np.block([[output_whole],[output]])
    return (output_whole)

Step 1: Crop the 1024 x 1024 images to 1024 pieces, each piece is 32 x 32 and Save

In [ ]:
Path_full_size = r'C:\Users\yizhe\Desktop\Velux\data\Change_plane\Full_size'
Path_crop = r'C:\Users\yizhe\Desktop\Velux\python\NN\Data\Predict_stitch\Crop_1024\image'
Predict_crop = r'C:\Users\yizhe\Desktop\Velux\python\NN\Data\Predict_stitch\Crop_1024\Predict'
N_pieces = 1024

In [ ]:
images = read_crop_write(Path_full_size, Path_crop, N_pieces)

Step 2: read the selected samples to prepare for prediction

In [ ]:
N_slices = N_pieces # for some reason it is not 1024, it only has 1 - 31
N_image_start = 2
N_image_stop = N_image_start

testGene = testGenerator(Path_crop,N_slices, N_image_start, N_image_stop)


Step 3: predict the image patch and Step 4: stitch the results

In [ ]:
from model import *

model = unet()
model.load_weights("unet_membrane.hdf5")
results = model.predict_generator(testGene, N_pieces, verbose = 1)
#saveResult(Predict_crop,results)
images = np.squeeze(results)
#np.shape(images)





whole_img = stitch_image (images, N_pieces)
whole_img = trans.resize(whole_img,[1024, 1024])

#plt.imshow(whole_img)

path = r'C:\Users\yizhe\Desktop\Velux\python\NN\Data\Predict_stitch\Crop_1024\Stitch'
filename = path + '\\' + str(N_image_start)+'.jpg'


cv2.imwrite(filename,whole_img*100)

In [ ]:
# create a loop to predict all the results

In [ ]:
for i in range(3, 52):
    N_image_start = i
    N_image_stop = N_image_start
    testGene = testGenerator(Path_crop,N_slices, N_image_start, N_image_stop)
    results = model.predict_generator(testGene, N_pieces, verbose = 1)
    images = np.squeeze(results)
    whole_img = stitch_image (images, N_pieces)
    whole_img = trans.resize(whole_img,[1024, 1024])
    
    filename = path + '\\' + str(N_image_start)+'.jpg'
    cv2.imwrite(filename,whole_img*100)




